# Creating snp database

In [ ]:
import sqlite3
from pathlib import Path

import pandas as pd
import pysam

from search_your_dna.snp_store import persist_all_snps_to_db, create_snp_db_schema, create_snp_db_chrom_pos_index, \
    create_snp_db_rsid_index

In [ ]:
bam_file_old = "/home/s/Dropbox/Siim/health/genetest_2020/GFX0237425.bam"
bam_file = "/home/s/Dropbox/Siim/health/genetest_2020/GFX0237425.GRCh38.p7.bam"
alignment_data_old = pysam.AlignmentFile(bam_file_old, "rb")
alignment_data = pysam.AlignmentFile(bam_file, "rb")

In [ ]:
snp_db_file = "/home/s/src/search_your_dna/data/ncbi_snpdb_all_ids.sqlite"
assert not Path(snp_db_file).exists(), "Warning! This is intended to be run only for populating a new database."

## Create SQL database

In [ ]:
create_snp_db_schema(snp_db_file=snp_db_file)


## get all SNP chr/pos values from ncbi

available for download in: https://ftp.ncbi.nih.gov/snp/organisms/human_9606/VCF/
as `00-All.vcf.gz`

## Store results in a sqlite db

In [ ]:
all_rsid_file = "/home/s/src/search_your_dna/data/00-All.vcf"
conn = sqlite3.connect(snp_db_file)

In [ ]:
%%time
persist_all_snps_to_db(conn, all_rsid_file)

## Build indices for fast lookups

In [ ]:
create_snp_db_chrom_pos_index(snp_db_file=snp_db_file)
create_snp_db_rsid_index(snp_db_file=snp_db_file)

## find my genotype for all SNP values


In [ ]:
snp_values = ["rs9380142", "rs587696773"]

